In [176]:
import requests
import os 
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [174]:
api_key = os.environ.get("stats_project")


location = 'bixi-toronto'
url = 'http://api.citybik.es/v2/networks/' + location


header_dict = {
    "accept": "application/json"
}

response = requests.get(url, headers=header_dict)

header_auth = {
    "accept": "application/json",
    "Authorization": api_key
}

counter = 0

for i in response.json()['network']['stations']:
        name = i['name']
        latitude = i['latitude']
        longitude = i['longitude']
        print("Name of Station: " + name + " Latitude : " + str(latitude) + " Longitude: " + str(longitude))
        print("=====================================")
        url = "https://api.foursquare.com/v3/places/search?ll={0}%2C{1}&radius=1000".format(latitude,longitude)
        response = requests.get(url, headers=header_auth)
        print(response.json())

    
        counter += 1
        if counter == 3:
            break

Name of Station: Summerhill Ave / Maclennan Ave Latitude : 43.685924 Longitude: -79.376304
{'results': [{'fsq_id': '4adcb343f964a520e32e21e3', 'categories': [{'id': 17069, 'name': 'Grocery Store', 'short_name': 'Grocery Store', 'plural_name': 'Grocery Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 77, 'geocodes': {'drop_off': {'latitude': 43.686221, 'longitude': -79.37519}, 'main': {'latitude': 43.686494, 'longitude': -79.375248}, 'roof': {'latitude': 43.686494, 'longitude': -79.375248}}, 'link': '/v3/places/4adcb343f964a520e32e21e3', 'location': {'address': '446 Summerhill Ave', 'country': 'CA', 'cross_street': 'btwn. MacLennan Ave. and Glen Rd.', 'formatted_address': '446 Summerhill Ave (btwn. MacLennan Ave. and Glen Rd.), Toronto ON M4W 2E4', 'locality': 'Toronto', 'postcode': 'M4W 2E4', 'region': 'ON'}, 'name': 'Summerhill Market', 'related_places': {'children': 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [175]:
data = response.json()
#print(data)

for key, values in data.items():
    if key == 'results':
        #print(key, value)
        for value in values:
            #print(value)
            for k,v in value.items():
                if k == 'categories': 
                    #print(v[0]['id'])
                    if 13026 <= v[0]['id'] <= 13390:
                        print(v[0]['name']) 
                if k == 'location':
                    print(v['formatted_address'])  
                if k == 'rating':
                    print(v)       
                    
                    

Burger Joint
3003 Danforth Ave (Victoria Park Ave), Toronto ON M4C 1M9
3003 Danforth Ave (at Victoria Park Ave), Toronto ON M4C 1M9
3003 Danforth Ave, Toronto ON M4C 1M9
Cafe, Coffee, and Tea House
3003 Danforth Ave (Victoria Park Avenue), East York ON M4C 1M9
Pizzeria
2980 Danforth Ave (Donlands Ave), Toronto ON M4C 1M6
3130 Danforth Ave (at Victoria Park Ave), Scarborough ON M1L 1B2
3111 Danforth Ave (at Victoria Park Ave), Toronto ON M1L 1A9
3003 Danforth Ave (at Victoria Park), Toronto ON M4C 1M9
Coffee Shop
50 Musgrave St, Toronto ON M4E 3W2
3003 Danforth Ave (at Victoria Park Ave.), Toronto ON M4C 1M9


Put your parsed results into a DataFrame

In [177]:
df = pd.json_normalize(data.items())
df.head()

""
0
1


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating